<h1><center>Predict Dota2 winner<center><h1>

[Соревнование](https://www.kaggle.com/c/mlcourse-dota2-win-prediction)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import pickle
from collections import defaultdict
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold, TimeSeriesSplit,\
                                    cross_val_score, cross_validate
import scipy.stats as stats
import gc
import warnings
warnings.filterwarnings('ignore')
import os
import category_encoders as ce
from tqdm import tqdm_notebook
import json
from itertools import combinations, groupby
from functools import reduce
from sklearn.cluster import KMeans
import time
from itertools import groupby
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
import lightgbm as lgb
from sklearn.feature_selection import RFECV
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV

In [2]:
# функция для чтения json файлов
def read_matches(path_to_file):
    with open(path_to_file) as fin:
        for line in fin: 
            yield json.loads(line)

In [3]:
# пути к файлам, размеры файлов
PATH_TR, TOTAL_TR = 'train_matches.jsonl', 39675
PATH_TE, TOTAL_TE = 'test_matches.jsonl', 10000


In [4]:
types = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    for obj in MATCH['objectives']:
        types.append(obj['type'])
unqiue_types = np.unique(types)
topkeys = list(MATCH.keys())

print('{:^25}'.format('teamfights'))
print('-----------------------')
for idx, el in enumerate(list(MATCH['teamfights'][0].keys())):
    print('{}) {}'.format(idx+1, el))
for idx2, el2 in enumerate(list(MATCH['teamfights'][0]['players'][0].keys())):
    print('{:>4}.{}) {}'.format(5, idx2+1, el2)) 
print('-----------------------\n\n')

print('{:^35}'.format('objectives'))
print('-----------------------------------')
for idx3, tp in enumerate(unqiue_types):
    print('{:>4}) {}'.format(idx3+1, tp))  
print('-----------------------------------\n\n')

print('{:^55}'.format('players'))
print('------------------------------------------------------------')
for k, v in MATCH['players'][0].items():
    print('{:>25} {}'.format(k, type(v)))
print('------------------------------------------------------------')


       teamfights        
-----------------------
1) start
2) end
3) last_death
4) deaths
5) players
   5.1) deaths_pos
   5.2) ability_uses
   5.3) item_uses
   5.4) killed
   5.5) deaths
   5.6) buybacks
   5.7) damage
   5.8) healing
   5.9) gold_delta
   5.10) xp_delta
-----------------------


            objectives             
-----------------------------------
   1) CHAT_MESSAGE_AEGIS
   2) CHAT_MESSAGE_AEGIS_STOLEN
   3) CHAT_MESSAGE_BARRACKS_KILL
   4) CHAT_MESSAGE_DENIED_AEGIS
   5) CHAT_MESSAGE_FIRSTBLOOD
   6) CHAT_MESSAGE_ROSHAN_KILL
   7) CHAT_MESSAGE_TOWER_DENY
   8) CHAT_MESSAGE_TOWER_KILL
-----------------------------------


                        players                        
------------------------------------------------------------
              player_slot <class 'int'>
                  hero_id <class 'int'>
                hero_name <class 'str'>
          account_id_hash <class 'str'>
         ability_upgrades <class 'list'>
               obs_placed <c

#### Целевой признак

In [5]:
target_L = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):    
    target_L.append((MATCH['match_id_hash'], MATCH['targets']['radiant_win']))
y_tr = pd.DataFrame.from_records(target_L).set_index(0)[1].astype(int).values

#### Глобальные статы

In [6]:
globals_L = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    globals_L.append((MATCH['game_time'], MATCH['match_id_hash'],
                      MATCH['game_mode'], MATCH['lobby_type']))
glob_df = pd.DataFrame.from_records(globals_L)

In [7]:
empty_df = pd.DataFrame(index = glob_df[1])

#### Чат (длина переписки, интесивность переписки)

In [8]:
# messages_L = []
# for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
#     for msg in MATCH['chat']:
#         try:
#             messages_L.append((MATCH['match_id_hash'], msg['player_slot']<100,\
#                                np.abs(msg['time']), msg['text'].count('*')))
#         except:
#             pass
# mess_df = pd.DataFrame.from_records(messages_L)
# mess_df[4] = mess_df[1].replace({True:1, False:-1}) * mess_df[3]

# X_tr = np.column_stack([glob_df.set_index(1),\
#                         empty_df.join(mess_df.groupby(0)[4].sum()),\
#                         empty_df.join(mess_df.groupby([0, 1])[2].apply(lambda x: np.diff(sorted(x)).mean()).unstack())])

In [9]:
X_tr = glob_df.set_index(1).values

#### Командные сражения

In [10]:
tf_D = defaultdict(list)
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']
    for tf_idx, tf in enumerate(MATCH['teamfights']):
        tf_D['glob'].append((idx, tf_idx, abs(tf['start']), abs(tf['end']), abs(tf['last_death']), tf['deaths']))
        for player_idx, player in enumerate(tf['players']):
            team = 'r' if player_idx<=4 else 'd'            
            for k, v in player['deaths_pos'].items():
                for k2,v2 in v.items():
                    tf_D['deaths_pos'].append((idx, team, k, k2, v2))
            for key in ('ability_uses', 'item_uses', 'killed'):
                for k, v in player[key].items():
                    tf_D[key].append((idx, team, k, v))
            for key in ('deaths', 'buybacks', 'damage', 'healing', 'gold_delta', 'xp_delta'):
                tf_D[key].append((idx, team, player[key]))
                
df_tf_glob = pd.DataFrame.from_records(tf_D['glob'])
df_tf_glob.columns = ['idx', 'tf_idx', 'start', 'end', 'last_death', 'deaths']
df_tf_glob['duration'] = df_tf_glob['end'] - df_tf_glob['start']
tf_glob_df = empty_df.join(df_tf_glob.groupby('idx').size().to_frame('count'))\
    .join(df_tf_glob.groupby('idx')['duration'].mean().to_frame('duration_mean'))

df_tf_glob['start2'] = df_tf_glob['start'] // 60
for ts in np.unique(df_tf_glob['start2']):
    tf_glob_df = tf_glob_df.join(df_tf_glob[df_tf_glob['start2']<=ts].groupby('idx').size().to_frame(ts))
tf_glob_df = tf_glob_df.join(df_tf_glob.groupby('idx').apply(lambda x: np.mean(x['deaths'])\
                                                / np.sum(x['duration'])).to_frame('death_per_min_tf'))
L2_tf = []
for key in tqdm_notebook(('deaths', 'buybacks', 'damage', 'healing', 'gold_delta', 'xp_delta')):
    _df = pd.DataFrame.from_records(tf_D[key])
    _ddf = empty_df.join(_df.groupby([0, 1])[2].sum().unstack())    
    L2_tf.append(_ddf.iloc[:, 1] - _ddf.iloc[:, 0])
for key in tqdm_notebook(('ability_uses', 'item_uses', 'killed')):
    _df = pd.DataFrame.from_records(tf_D[key])
    _df[4] = _df[1].replace({'r':1, 'd':-1}) * _df[3]
    L2_tf.append(empty_df.join(_df.groupby([0, 2])[4].sum().unstack()))
    
tf_glob_df = pd.concat([tf_glob_df, pd.concat(L2_tf, 1)], 1)
_df = pd.DataFrame.from_records(tf_D['deaths_pos'])
_df[5] = _df[2].astype(str) +'_' + _df[3].astype(str)
_df[6] = _df[4] * _df[1].replace({'r':1, 'd':-1})

X_tr = np.column_stack([X_tr, empty_df.join(_df.groupby([0, 5])[6].sum().unstack())])

#### Обжективы (глобальные объекты на карте) (собираем мешок обжективов)

In [26]:
obj_L = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for obj in MATCH['objectives']:
        obj['idx'] = idx
        obj_L.append(obj)
df = pd.DataFrame.from_records(obj_L)

df.loc[df['team'].isna(), 'flag'] = df[df['team'].isna()]['player_slot'].apply(lambda x: 2 if x<100 else 3)
df.loc[~df['team'].isna(), 'flag'] = df[~df['team'].isna()]['team']
df.loc[df['flag'].isin([60, 100]), 'flag'] = df[df['flag'].isin([60, 100])]['slot'].apply(lambda x: 2 if x<=4 else 3)
df['flag'] = df['flag'].replace({2:1, 3:-1})

L2 = []
for ts in tqdm_notebook(np.unique((df['time']/60).round())):
    _subdf = df[df['time']/60 <= ts]
    L2.append(empty_df.join(_subdf.groupby(['idx', 'type'])['flag'].sum().unstack()))
    L2.append(empty_df.join(_subdf.groupby(['idx', 'type'])['flag'].mean().unstack()))
    
X_tr = np.column_stack([X_tr, np.column_stack(L2)])

In [27]:
X_tr.shape

(39675, 4049)

#### Игроки (конечные числовые характеристики)

In [28]:
numbers = ['assists', 'camps_stacked', 'creeps_stacked', 'deaths', 'denies',
           'firstblood_claimed', 'gold', 'health',  'kills',
           'level', 'lh', 'max_health', 'max_mana',
           'nearby_creep_death_count', 'obs_placed', 'observers_placed',
           'roshans_killed','rune_pickups', 'sen_placed', 'stuns', 'teamfight_participation',
           'towers_killed', 'xp']

D_players = defaultdict(list)
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for key in numbers:
            D_players[key].append((idx, team, player[key]))
L3 = []
for k, v in tqdm_notebook(D_players.items()):
    _df = pd.DataFrame.from_records(v)
    L2 = []
    for idx, _subdf in tqdm_notebook(_df.groupby(0)):
        _subdf_r = _subdf[_subdf[1] == 'r'][2]
        _subdf_d = _subdf[_subdf[1] == 'd'][2]
        diffs = []
        for vr in _subdf_r.values:
            for vd in _subdf_d.values:
                diffs.append(vr-vd)
        diffs = sorted(diffs)
        L2.append([idx] + diffs)
    L3.append(empty_df.join(pd.DataFrame.from_records(L2).set_index(0)))
    
X_tr = np.column_stack([X_tr, np.column_stack(L3)])

#### Игроки (герои, координаты)

In [46]:
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for key in ('hero_id','x', 'y'):
            D_players[key].append((idx, team, player[key]))
            
hero_coords_L = [] 
for key in ('hero_id','x', 'y'):
    _df = pd.DataFrame.from_records(D_players[key])
    _df[1] = _df[1].replace({'r':1, 'd':-1})
    hero_coords_L.append(empty_df.join(_df.groupby([0, 2])[1].sum().unstack())) 
    
X_tr  = np.column_stack([X_tr, np.column_stack(hero_coords_L)])
del hero_coords_L
gc.collect()

4

#### Игроки (золото, ласт хиты, денаи, опыт с течением времени)

In [47]:
ts_keys = ['gold_t', 'xp_t', 'lh_t', 'dn_t']
ts_D = defaultdict(list)
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for key in ts_keys:
            for element_idx, element in enumerate(player[key]):
                ts_D[key].append((idx, team, element_idx, element))
                
ts_L = []
for k, v in tqdm_notebook(ts_D.items()):    
    _df = pd.DataFrame.from_records(v)
    _df[1] = _df[1].replace({'r':1, 'd':-1})
    _df[3] = _df[1] * _df[3]
    _df = _df.drop(1, axis = 1)
    _ddf = _df.groupby([0, 2])[3].mean().unstack()
    
    _L = []
    for i in tqdm_notebook(range(len(_ddf))):
        ser = _ddf.iloc[i].dropna()
        from itertools import groupby
        L = []
        for k, v in groupby(ser.values, key = lambda x: x<0):
            L.append((k, len(list(v))/len(ser)))
        L_ser = pd.DataFrame.from_records(L).groupby(0)[1].max()
        try:
            fs = L_ser[False]
        except:
            fs = np.nan
        try:
            tr = L_ser[True]
        except:
            tr = np.nan

        _L.append((ser.name, (ser < 0).mean(), ser.diff().dropna().mean(), tr, fs))
    ts_L.append(empty_df.join(pd.DataFrame.from_records(_L).set_index(0)        ))
X_tr  = np.column_stack([X_tr,np.column_stack(ts_L)])
del ts_D, ts_L
gc.collect()

25

#### Игроки (апгрейд умений)

In [48]:
abil_upgrades_L = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for upgrade in player['ability_upgrades']:
            upgrade['idx'] = idx
            upgrade['team'] = team
            abil_upgrades_L.append(upgrade)
_df = pd.DataFrame.from_records(abil_upgrades_L)
_df['time'] = _df['time'].abs()
X_tr  = np.column_stack([X_tr, empty_df.join(_df.groupby(['idx', 'team'])['time']\
                                                .apply(lambda x: np.diff(sorted(x)).mean()).unstack())])

#### Игроки (buyback:выкуп мертвого героя)

In [50]:
bb_log_L = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for bb_log in player['buyback_log']:
            bb_log_L.append((idx, ('r' if bb_log['player_slot'] < 100 else 'd'), np.abs(bb_log['time'])))
_df = pd.DataFrame.from_records(bb_log_L)
X_tr  = np.column_stack([X_tr,empty_df.join(_df.groupby([0, 1])[2].apply(lambda x: np.diff(sorted(x)).mean()).unstack())])
del bb_log_L, _df
gc.collect()

4

#### Игроки (hero inventory: инвентарь героя)

In [51]:
inv_L = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for item in player['hero_inventory']:
            try:
                inv_L.append((idx, team, item['id'], item['cooldown']))
            except:
                pass
_df = pd.DataFrame.from_records(inv_L)  
_df[4] = _df[3] * _df[1].replace({'r':1, 'd':-1})
X_tr  = np.column_stack([X_tr, empty_df.join(_df.groupby([0, 2])[4].sum().unstack()),\
                               empty_df.join(_df.groupby([0, 2])[4].mean().unstack())])
del inv_L, _df
gc.collect()

17

#### Игроки (kills log: логи убийств)

In [52]:
kl_L = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for log in player['kills_log']:
            kl_L.append((idx, team, np.abs(log['time'])))
            
_df = pd.DataFrame.from_records(kl_L)  
X_tr  = np.column_stack([X_tr, empty_df.join(_df.groupby([0, 1])[2].apply(lambda x: np.diff(sorted(x)).mean()).unstack())])
del kl_L, _df
gc.collect()

4

#### Игроки (purchase log: логи покупок)

In [53]:
pc_L = []
rn_L = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for log in player['purchase_log']:
            pc_L.append((idx, team, np.abs(log['time']), log['key']))
        for log in player['runes_log']:
            rn_L.append((idx, team, np.abs(log['time']), log['key']))
_df = pd.DataFrame.from_records(pc_L) 
_df[1] = _df[1].replace({'r':1, 'd':-1})
X_tr  = np.column_stack([X_tr, empty_df.join(_df.groupby([0, 3])[1].sum().unstack()),\
                               empty_df.join(_df.groupby([0, 3])[1].mean().unstack())])

_df = pd.DataFrame.from_records(rn_L) 
_df[1] = _df[1].replace({'r':1, 'd':-1})
X_tr  = np.column_stack([X_tr, empty_df.join(_df.groupby([0, 3])[1].sum().unstack()),\
                               empty_df.join(_df.groupby([0, 3])[1].mean().unstack())])

del pc_L, _df, rn_L
gc.collect()


4

#### Игроки (логи вардов)

In [54]:
D = defaultdict(list)
logs = ['obs_left_log', 'obs_log', 'sen_left_log', 'sen_log']
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for key in logs: 
            for element in player[key]:
                D[key].append((idx, team, str(element['x'] // 10)+'_'+str(element['y'] // 10)+'_'+str(element['z'] // 10)))
            
            
_logs_L = []
for k, v in tqdm_notebook(D.items()):
    _df = pd.DataFrame.from_records(v)
    _df[1] = _df[1].replace({'r':1, 'd':-1})
    _logs_L.append(empty_df.join(_df.groupby([0, 2])[1].sum().unstack()))  
    _logs_L.append(empty_df.join(_df.groupby([0, 2])[1].mean().unstack()))  
    
X_tr  = np.column_stack([X_tr, np.column_stack(_logs_L)])
del D, _logs_L, _df
gc.collect()

79

#### Игроки (использованные умения, айтемы)

In [55]:
abil_uses_L = []
item_uses_L = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for k, v in player['ability_uses'].items():
            abil_uses_L.append((idx, team, k, v))
        for k, v in player['item_uses'].items():
            item_uses_L.append((idx, team, k, v))
_df = pd.DataFrame.from_records(abil_uses_L)
_df[1] = _df[1].replace({'r':1, 'd':-1})
_df[4] = _df[1]*_df[3]

_df2 = pd.DataFrame.from_records(item_uses_L)
_df2[1] = _df2[1].replace({'r':1, 'd':-1})
_df2[4] = _df2[1]*_df2[3]

X_tr = np.column_stack([X_tr, np.column_stack([empty_df.join(_df.groupby([0, 2])[4].sum().unstack()),\
                                               empty_df.join(_df2.groupby([0, 2])[4].sum().unstack()),\
                                               empty_df.join(_df.groupby([0, 2])[4].mean().unstack()),\
                                               empty_df.join(_df2.groupby([0, 2])[4].mean().unstack())])])        

#### Игроки (действия, причины получения золотта, серии убийств, мульти убийства, пинги, причины получения опыта)

In [56]:
LL = ['actions', 'gold_reasons', 'kill_streaks', 'life_state', 'multi_kills', 'pings', 'xp_reasons']
D = defaultdict(list)
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']  
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'
        for key in LL:
            for k, v in player[key].items():
                D[key].append((idx, team, k, v))
_dfs = []
for k, v in D.items():
    _df = pd.DataFrame.from_records(v)
    _df[4] = _df[1].replace({'r':1, 'd':-1}) * _df[3]
    _dfs.append(empty_df.join(_df.groupby([0, 2])[4].sum().unstack()))
    _dfs.append(empty_df.join(_df.groupby([0, 2])[4].mean().unstack()))
    
X_tr = np.column_stack([X_tr,np.column_stack(_dfs)])
del D, _dfs, _df
gc.collect()

4

#### Игроки (урон)

In [57]:
dmg_L = []
for MATCH in tqdm_notebook(read_matches(PATH_TR), total = TOTAL_TR):
    idx = MATCH['match_id_hash']
    for player_idx, player in enumerate(MATCH['players']):
        team = 'r' if player_idx<=4 else 'd'   
        for k, v in player['damage'].items():
            dmg_L.append((idx, team, k, v))
_df = pd.DataFrame.from_records(dmg_L)  

_df[4] = _df[1].replace({'r':1, 'd':-1}) * _df[3]
X_tr = np.column_stack([X_tr,empty_df.join(_df.groupby([0, 2])[4].sum().unstack()),\
                             empty_df.join(_df.groupby([0, 2])[4].mean().unstack())])

In [59]:
print('Кол-во признаков = {}'.format(X_tr.shape[1]))

Кол-во признаков = 9008


In [60]:
# with open('X_tr.pickle', 'wb') as f:
#     pickle.dump(X_tr, f)
# with open('y_tr.pickle', 'wb') as f:
#     pickle.dump(y_tr, f)

In [5]:
# with open('X_tr.pickle', 'rb') as f:
#     X_tr = pickle.load(f)
# with open('y_tr.pickle', 'rb') as f:
#     y_tr = pickle.load(f)

#### Отбираем признаки

In [63]:
def _select_features(estimator, X, y, cv, scoring, holdout_share, stratify, seed):
    
    '''Рекурсивно отбирает признаки (в соответствии с метриками на валидационной и на отложенной выборках)'''
    
    ############################################################################################################
    def _get_val_holdout_scores(estimator, _X_TR, _X_HOLDOUT, Y_TR, Y_HOLDOUT, cv, metric):
        mean_cv = cross_validate(estimator, _X_TR, Y_TR, cv = cv, scoring = metric)['test_score'].mean()
        estimator.fit(X_TR[:, current_features], Y_TR)
        Y_PREDPROB = estimator.predict_proba(_X_HOLDOUT)[:, 1]
        Y_PRED = estimator.predict(_X_HOLDOUT)
        if metric == 'roc_auc':
            holdout = roc_auc_score(Y_HOLDOUT, Y_PREDPROB)
        if metric == 'accuracy':
            holdout = accuracy_score(Y_HOLDOUT, Y_PRED)
        if metric == 'precision':
            holdout = precision_score(Y_HOLDOUT, Y_PRED)
        if metric == 'recall':
            holdout = recall_score(Y_HOLDOUT, Y_PRED)
        if metric == 'f1':
            holdout = f1_score(Y_HOLDOUT, Y_PRED)
        return (mean_cv, holdout)
    #################################################################################################################
    
    try:
        _X, _y = np.array(X), np.array(y)
    except:
        _X, _y = X, y
        
    if stratify:
        # разбиваем данные на трейн и отложенную части, сохраняя пропорции таргета
        X_TR, X_HOLDOUT, Y_TR, Y_HOLDOUT = train_test_split(_X, _y, stratify = _y,\
                                                            test_size = holdout_share,\
                                                            random_state = seed, shuffle = True)
    else:
        X_TR, X_HOLDOUT, Y_TR, Y_HOLDOUT = train_test_split(_X, _y, test_size = holdout_share)
        
    # считаем валидацию каждого признака
    scores = []
    for i in tqdm_notebook(range(X_TR.shape[1])):
        mean_cv = cross_validate(estimator, X_TR[:, i].reshape(-1,1), Y_TR, cv = cv, scoring = scoring)['test_score'].mean()
        scores.append(mean_cv)

    # сортируем признаки в порядке убывания валидации
    order = np.argsort(scores)[::-1]

    # лучший набор, лучшая валидация, лучшая отложенная
    # бесполезные признаки до и после повторного отбора 
    best_features, best_cv, best_holdout, to_drop_before, to_drop_after = [], .5, .5, [], []

    # по признакам в порядке убывания валидации
    for i in tqdm_notebook(order):

        # добавляем к писку лучших
        current_features = best_features+[i]

        # валидация, отложенная
        mean_cv, holdout = _get_val_holdout_scores(estimator, X_TR[:, current_features], X_HOLDOUT[:, current_features],\
                                                   Y_TR, Y_HOLDOUT, cv=cv, metric=scoring)

        # если оба скора растут
        if (mean_cv>best_cv) & (holdout>best_holdout):
            # обновляем лучшие скоры, лучшие признаки
            best_cv = mean_cv
            best_holdout = holdout
            best_features = current_features
        else:
            # если скоры не растут, добавляем признак в претенденты на удаление
            to_drop_before.append(i)

    # пока списки кандидатов на удаление ДО и ПОСЛЕ отбора не перестанут изменяться
    while True:   

        # добавляем очередной признак
        for i in tqdm_notebook(to_drop_before):
            current_features = best_features+[i]
            
            # валидация, отложенная
            mean_cv, holdout = _get_val_holdout_scores(estimator, X_TR[:, current_features], X_HOLDOUT[:, current_features],\
                                                       Y_TR, Y_HOLDOUT, cv=cv, metric=scoring)
            
            if (mean_cv>best_cv) & (holdout>best_holdout):
                best_cv = mean_cv
                best_holdout = holdout
                best_features = current_features
            else:
                # выбрасываем, если он не дал улучшений
                to_drop_after.append(i)

        # если списки признаков до и после отбора не изменились, прекращаем отбор
        if len(to_drop_before) == len(to_drop_after):
            break
        else:
            # список до --> список после, список после - пустой
            to_drop_before = to_drop_after
            to_drop_after = []   
            
    return (best_cv, best_holdout, best_features)
    

In [64]:
SEED = 13
best_cv, best_holdout, best_features = _select_features(estimator=LGBMClassifier(random_state = SEED),\
                                                        X=X_tr, y=y_tr,\
                                                        cv=StratifiedKFold(3, random_state = SEED),\
                                                        scoring='roc_auc',\
                                                        holdout_share=.2, stratify=True, seed=SEED)

In [65]:
X_selected_tr = X_tr[:, best_features]

In [66]:
print('auc на валидации = {}\nauc на отложенной = {}'.format(best_cv, best_holdout))

auc на валидации = 0.8295071267301249
auc на отложенной = 0.8330917300607246


In [68]:
# with open('X_selected_tr.pickle', 'wb') as f:
#     pickle.dump(X_selected_tr, f)

# with open('X_selected_tr.pickle', 'rb') as f:
#     X_selected_tr = pickle.load(f)

In [51]:
%%time

lgb_params = {'n_estimators': [500], 'learning_rate':[.05],\
             'max_depth':[3, 5, 7, 70, 90, 110],\
             'num_leaves' : [32, 64, 128, 256, 512],
             'min_child_samples': [50, 100, 200, 300]}

lgb_grid = GridSearchCV(LGBMClassifier(random_state = SEED), param_grid = lgb_params,\
              scoring = 'roc_auc', cv = StratifiedKFold(3, random_state=SEED))

lgb_grid.fit(X_selected_tr, y_tr)

lgb_params2 = {'learning_rate':np.linspace(.005, .1, 500)}
lgb_grid2 = GridSearchCV(LGBMClassifier(random_state = SEED, **lgb_grid.best_params_), param_grid = lgb_params2,\
              scoring = 'roc_auc', cv = StratifiedKFold(3, random_state=SEED))

lgb_grid2.fit(X_selected_tr, y_tr)

Wall time: 33min 18s


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=13, shuffle=False),
             estimator=LGBMClassifier(learning_rate=0.05, max_depth=3,
                                      min_child_samples=50, n_estimators=500,
                                      num_leaves=32, random_state=13),
             param_grid={'learning_rate': array([0.005     , 0.00519038, 0.00538076, 0.00557114, 0.00576152,
       0.0059519 , 0.00614228, 0.00633267, 0.00652305, 0.00671343,
       0.00690381, 0...
       0.09352705, 0.09371743, 0.09390782, 0.0940982 , 0.09428858,
       0.09447896, 0.09466934, 0.09485972, 0.0950501 , 0.09524048,
       0.09543086, 0.09562124, 0.09581162, 0.096002  , 0.09619238,
       0.09638277, 0.09657315, 0.09676353, 0.09695391, 0.09714429,
       0.09733467, 0.09752505, 0.09771543, 0.09790581, 0.09809619,
       0.09828657, 0.09847695, 0.09866733, 0.09885772, 0.0990481 ,
       0.09923848, 0.09942886, 0.09961924, 0.09980962, 0.1       ])},
             scoring='roc_auc')

In [59]:
best_params_d = lgb_grid.best_params_
best_params_d.update(lgb_grid2.best_params_)

print('auc (lightgbm с подбором гиперпараметров) = {}'.format(lgb_grid2.best_score_))
print('оптимальные гиперпараметры:')
for k, v in best_params_d.items():
    print('\t{} = {}'.format(k, v))

auc (lightgbm с подбором гиперпараметров) = 0.8322006793470723
оптимальные гиперпараметры:
	learning_rate = 0.06211422845691383
	max_depth = 3
	min_child_samples = 50
	n_estimators = 500
	num_leaves = 32
